In [77]:
from __future__ import print_function, division 
import time 
 
import tarfile

import sys

import numpy as np 
import os

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 

from PIL import Image

import cv2

import pandas as pd

from tensorflow.keras import Model, Sequential  
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation, ZeroPadding2D, UpSampling2D, Reshape, BatchNormalization, Flatten, Dropout  
from tensorflow.keras.layers import LeakyReLU 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers import Adam 

GAN class for image generation which mimic the X_train, an array of imgs.

In [192]:
class GAN():
    def __init__(self):
        self.img_rows = 200
        self.img_cols = 360
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        print(self.img_cols)
        
        optimizer = Adam(0.002,0.5)
        
        #Build and compile the discriminator 
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        #Build the generator
        self.generator = self.build_generator()
        
        #The generator takes noise as input and generates images 
        z = Input(shape=(self.latent_dim))
        img = self.generator(z)
        
        # For the combined model we will only train the generator 
        self.discriminator.trainable = False
        validity = self.discriminator(img)
        
        #Combined model (stacked generator and discriminator)
        #Train generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_shape = (self.latent_dim,)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))
        print('Model summary for Generator')
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation = 'sigmoid'))
        print('Model summary for Discriminator')
        model.summary()
        
        img = Input(shape=self.img_shape)
        validity = model(img)
        # discriminator ouputs two entries, one validity and one accuracy matrix
        return Model(img, validity)
    
    def train(self, epochs, batch_size=2, sample_interval=50):
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            
            # ---------------------
            #  Train Discriminator
            # ---------------------
            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            
            noise = np.random.normal(0,1,(batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)
            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            # d_loss consist of two entry, first one loss, second one accuracy matrix
            d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            # ---------------------
            #  Train Generator
            # ---------------------
            # Train generator, such that the discriminator gets fooled
            noise = np.random.normal(0,1,(batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise,valid)
            # plot progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
            
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r*c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5*gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:])
                axs[i,j].axis('off')
                cnt += 1
        # the directory here is wrong
        fig.savefig("/Users/yuf/Desktop/Implementation/GAN/Trained_Bab/%d.png" % epoch)
        plt.close()

In [193]:
gan_ksgk = GAN()

360
Model summary for Discriminator
Model: "sequential_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 216000)            0         
_________________________________________________________________
dense_126 (Dense)            (None, 512)               110592512 
_________________________________________________________________
leaky_re_lu_83 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_127 (Dense)            (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_84 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_128 (Dense)            (None, 1)                 257       
Total params: 110,724,097
Trainable params: 110,724,097
Non-trainable params: 0
___

In [ ]:
gan_ksgk.train(epochs=1000, batch_size=4, sample_interval=50)